## Module 6 Practice - Choropleth Maps


In this notebook, we will see examples of **choropleth maps** using ggplot.

Maps are represented by very **detailed polygons in vector form** as we have seen in the [L6 Projections Practice](L6_Projections_Practice.ipynb). 


First, let's start with simple polygons and see how to define and draw them. 

To color polygons, we usually need **two data frames**: 

 - one contains the **coordinates** of each polygon (positions),  
 
 - and the other one contains the **values** associated with each polygon (values).  
 
A **common variable** (id) links the two together. 


In [ ]:
library(ggplot2)
library(maps)
library(ggmap)
library(mapproj)

In [ ]:
# Some made up IDs; these connect values to polygons
ids <- factor(c("1.1", "2.1", "1.2", "2.2", "1.3", "2.3"))

# Some made up values: the colors will represent these values 
values <- data.frame(some_column = ids, value = c(3, 3.1, 3.1, 3.2, 3.15, 3.5))

# coordinates of some polygons
positions <- data.frame(id = rep(ids, each = 4),
  x = c(2, 1, 1.1, 2.2, 1, 0, 0.3, 1.1, 2.2, 1.1, 1.2, 2.5, 1.1, 0.3,
  0.5, 1.2, 2.5, 1.2, 1.3, 2.7, 1.2, 0.5, 0.6, 1.3),
  y = c(-0.5, 0, 1, 0.5, 0, 0.5, 1.5, 1, 0.5, 1, 2.1, 1.7, 1, 1.5,
  2.2, 2.1, 1.7, 2.1, 3.2, 2.8, 2.1, 2.2, 3.3, 3.2))

head(values)
head(positions)

**Let's draw and color them:** `map_id` is what associates the two data frames. `geom_map` expects to have either (long,lat,group) or (x,y,id) variable names in the map data. Here, we have the latter. 

In [ ]:
ggplot(data=values, aes(fill=value)) +

geom_map(map=positions, aes(map_id=some_column)) +

expand_limits(positions)


We associated the map and the values by telling `geom_map` the column `some_column` in the data frame is same as the `id` (or `region`) column in the map data frame. 

---

**Now, let's do the same for US map to visualize the crime data set.** 

**Important:** We will use state name as the **common variable** so they have to match to the state names in the map data.

In [ ]:
head(USArrests)

First create **lowercase state names**, because these will be the "IDs" that will connect 
the crimes data to the map data. 


Since map data also contains lowercase state names and their polygon coordinates, **we will use the lowercase state name as the common ID.** 

In [ ]:
crimes <- data.frame(state = tolower(rownames(USArrests)), USArrests)

In [ ]:
# convert arrest variables in the columns to a single column to enable faceting later. 
crimesm <- reshape2::melt(crimes[,c(1,2,3,5)], id = 1)
head(crimes)
head(crimesm)

In [ ]:
# Get the map data for US states
states_map <- map_data("state")

In [ ]:
head(states_map)

In [ ]:
# color will encode Murder 
gg <- ggplot(data=crimes) + 

# "state" is the common variable, associate it to "region" variable in the map 
geom_map(map=states_map, aes(map_id=state, fill=Murder), color="white", size=0.1) +

# coordinate limits of the map 
expand_limits(x=states_map$long, y=states_map$lat) + 

# useful map projection 
coord_map("polyconic") +

# remove grid and coord info 
theme_void() 

gg

**Now this is a choropleth map.** 


We can also do small multiples of maps like this: 

In [ ]:
ggplot(data=crimesm, aes(fill=value)) +

geom_map(map=states_map, aes(map_id = state), color="white", size=0.1) +

expand_limits(x = states_map$long, y = states_map$lat) +

facet_wrap(~variable, ncol=2) + 

coord_map("polyconic") +

theme_void()

### YOUR TURN:

**Create a choropleth map for population using the crimes data set. Use `YlOrRd` palette from brewer library and make sure that DARKER colors represent LARGER population numbers.**


(Hint: You'll have to do some digging on Internet to figure out how to use the `YlOrRd` for a continuous variable in the right order. Start by looking at `scale_fill_` functions.)

In [ ]:
#< YOUR CODE HERE >


---

**Let's get another data set and work on it.** 

In [ ]:
agex <- read.csv("/dsa/data/all_datasets/2011_us_ag_exports.csv")
head(agex)

In [ ]:
library(dplyr) 
# Let's create lowercase state names 
agex <- mutate(agex, state = tolower(state))
head(agex)

In [ ]:
# Let's draw and color them
g_ex <- ggplot() +

 geom_map(data=agex, map=states_map, aes(fill=total.exports, map_id=state), color="white", size=0.3) +

 expand_limits(x = states_map$long, y = states_map$lat) +

 coord_map("polyconic") +

 theme_void() +

 ggtitle("US Agriculture Exports") 

g_ex

We may prefer to represent the highest number by the darkest color. So, we need to change the direction of the palette: 

In [ ]:
g_ex + scale_fill_continuous(trans="reverse")

In [ ]:
# Pick a few variables from the data set 
sub <- agex[, c(2,14,15,16,17)]
head(sub)
# melt them into a single variable
subm <- reshape2::melt(sub, id = 1)
head(subm)

In [ ]:
ggplot(subm) +

geom_map(map=states_map, aes(map_id=state, fill=value),  color="white", size=0.1) +

scale_fill_continuous(low='thistle2', high='darkblue') +

facet_wrap(~variable, ncol=2) +

expand_limits(x = states_map$long, y = states_map$lat) + 
coord_map("polyconic") + 
theme_void() +
#theme(legend.position="None") + 
ggtitle("US Agriculture Exports") 


**Let's compare the percentage contributions of states for each product instead of actual numbers. It might be more or less meaningful depending on the visual task.**

In [ ]:
subp <- sub
subp <-mutate(subp, corn=corn/sum(corn))
subp <-mutate(subp, wheat=wheat/sum(wheat))
subp <-mutate(subp, cotton=cotton/sum(cotton))
subp <-mutate(subp, total.veggies=total.veggies/sum(total.veggies))
subpm <- reshape2::melt(subp, id = 1)

### YOUR TURN:  

**Copy and paste the small multiples plotting code from above and run. Compare the colors of both maps.** 

 - **Which one seems to be more truthful to the data ?**
 - What kind of scales should we use for color palettes to make it easier for comparison, but also represent the data truthfully? 
 

In [ ]:
#copy code here and run 



If there is a dominating value in a data set, we can apply a log or sqrt transform to our palette to better show the variation in the smaller values. 

In [ ]:
ggplot(subm) +
geom_map(map=states_map, aes(map_id=state, fill=value),  color="white", size=0.1) +
expand_limits(x = states_map$long, y = states_map$lat) + 
scale_fill_continuous(low='thistle2', high='darkblue', trans="sqrt") +
coord_map("polyconic") + 
facet_wrap(~variable, ncol=2) +
theme_void() +
#theme(legend.position="None") + 
ggtitle("US Agriculture Exports") 

**Is this better or truthful?**

**Choosing color palettes, transforms, and limits for choropleth maps is tricky; since human visual system cannot efficiently make direct comparisons of color shades, we can easily undermine or exaggerate the representation of the values in the choropleth maps.**

